In [2]:
from utils.df_handle import *

In [277]:
# day_ago = 2
# datenow = datetime.now().strftime("%Y%m%d")
# datenow_day_ago = ( datetime.now()-timedelta(day_ago) ).strftime("%Y%m%d")
# param_1 = f"'{datenow_day_ago}'"
datenow = datetime.now().strftime("%Y%m%d")
param_0 = f"'20210701'"
param_1 = f"'20211001'"
param_2 = f"'20211024'"
# param_2 = f"'{datenow}'"
param_3 = f"'{datenow}'"
# param_2 = f"'20210501'"
# param_3 = f"'20210930'"

In [278]:
# Extract csv

wcc = \
f"""
SELECT BranchID,
       NumberCICO
--INTO #WithOutNumberCICO
FROM dbo.AR_SalespersonLocationTrace WITH (NOLOCK)
WHERE Type LIKE '%DE%'
      OR Type LIKE '%PA%'
         AND CAST(UpdateTime AS DATE)
      BETWEEN {param_1} AND {param_2};
"""
get_ms_csv(wcc, 'WithOutNumberCICO.csv')

rd = \
f"""
SELECT 
CustID, SlsFreq, DayofWeek, SalesRouteID, VisitDate
--INTO #T_RouteDet
FROM dbo.OM_SalesRouteDet d WITH (NOLOCK)
WHERE d.VisitDate
BETWEEN {param_1} AND {param_2};
"""
get_ms_csv(rd, 'T_RouteDet.csv')

srm = "SELECT BranchID, SalesRouteID, SlsperID, FromDate, ToDate FROM dbo.Vs_SalespersonRoute"
get_ms_csv(srm, 'SRM.csv')

slr = "SELECT CustID, SalesRouteID, StartDate, EndDate, SlsFreq FROM dbo.vs_OM_SalesRouteMaster"
get_ms_csv(slr, 'SLR.csv')

stl = f"SELECT BranchID, SlsperID, CustID, UpdateTime, NumberCICO FROM dbo.AR_SalespersonLocationTrace WHERE CAST(UpdateTime AS DATE) BETWEEN {param_1} AND {param_2}"
get_ms_csv(stl, 'STL.csv')

pso = f"""SELECT
BranchID, SlsPerID, SalesRouteID, CustID, Crtd_DateTime, InsertFrom, LineAmt
FROM dbo.OM_PDASalesOrd WHERE CAST(Crtd_DateTime AS DATE) BETWEEN {param_1} AND {param_2} and Status = 'C' and OrderType = 'IN'"""
get_ms_csv(pso, 'PSO.csv')

ts = "SELECT BranchID, SlsperID, SupID, ASM, RSMID FROM dbo.fr_ListSaleByData('Admin')"
get_ms_csv(ts, 'TSlsperID.csv')
# INTO #TSlsperID
u = """SELECT UserName, FirstName, Position from Users"""
get_ms_csv(u, 'Users.csv')

com = """SELECT CpnyID, CpnyName from dbo.SYS_Company"""
get_ms_csv(com, 'COM.csv')

cu = """SELECT CustId, CustName, Channel, ShopType, HCOID, HCOTypeID, ClassId, TerritoryDescr, StateDescr, DistrictDescr, WardDescr from dbo.vs_AR_CustomerInfo"""
get_ms_csv(cu, 'CU.csv')

In [279]:
# Get dataframe
SRM = pd.read_csv('SRM.csv', dayfirst=True, parse_dates=['FromDate', 'ToDate'])

SLR = pd.read_csv('SLR.csv', dayfirst=True, parse_dates=['StartDate', 'EndDate'])

RD = pd.read_csv('T_RouteDet.csv')
RD.VisitDate = convert_to_datetime(RD.VisitDate)

STL = pd.read_csv('STL.csv')
STL['UpdateTime']=convert_to_datetime(STL['UpdateTime'])

PSO = pd.read_csv('PSO.csv')
PSO.Crtd_DateTime = convert_to_datetime(PSO.Crtd_DateTime)

WCC = pd.read_csv('WithOutNumberCICO.csv')

TS = pd.read_csv('TSlsperID.csv')

U = pd.read_csv('Users.csv')

COM = pd.read_csv('COM.csv')

CU = pd.read_csv('CU.csv')

In [280]:
RD.columns

Index(['CustID', 'SlsFreq', 'DayofWeek', 'SalesRouteID', 'VisitDate'], dtype='object')

In [281]:
SRM.columns

Index(['BranchID', 'SalesRouteID', 'SlsperID', 'FromDate', 'ToDate'], dtype='object')

In [282]:
SLR.columns

Index(['CustID', 'SalesRouteID', 'StartDate', 'EndDate', 'SlsFreq'], dtype='object')

In [283]:
# Get ET

SLR['SalesRouteID'] = SLR['SalesRouteID'].str.upper()
RD['SalesRouteID'] = RD['SalesRouteID'].str.upper()

T_ExtRoute = RD.merge(SRM.add_prefix('SRM_'), how='inner', left_on=['SalesRouteID'], right_on='SRM_SalesRouteID')
T_ExtRoute = df_filter(T_ExtRoute, DayofWeek='Sun')
T_ExtRoute=T_ExtRoute[\
    (T_ExtRoute['VisitDate'] >= T_ExtRoute['SRM_FromDate']) & \
    (T_ExtRoute['VisitDate'] <= T_ExtRoute['SRM_ToDate'])]

T_ExtRoute = T_ExtRoute.merge(SLR.add_prefix('SLR_'), how='inner', left_on=['SalesRouteID', 'CustID'], right_on=['SLR_SalesRouteID', 'SLR_CustID'])
T_ExtRoute=T_ExtRoute[\
    (T_ExtRoute['VisitDate'] >= T_ExtRoute['SLR_StartDate']) & \
    (T_ExtRoute['VisitDate'] <= T_ExtRoute['SLR_EndDate'])]

T_ExtRoute = dropdup(T_ExtRoute, 1, ['SRM_BranchID', 'CustID', 'SRM_SlsperID', 'SlsFreq', 'SLR_StartDate', 'SLR_EndDate'])
T_ExtRoute = T_ExtRoute[['SRM_BranchID','CustID','SRM_SlsperID','SlsFreq','SLR_StartDate','SLR_EndDate']]
T_ExtRoute.columns = ['BranchID', 'CustID', 'SlsperID', 'SlsFreq', 'StartDate','EndDate']
print(T_ExtRoute.shape)

(6341, 6)


In [284]:
T_ExtRoute.columns

Index(['BranchID', 'CustID', 'SlsperID', 'SlsFreq', 'StartDate', 'EndDate'], dtype='object')

srm.BranchID,
rd.CustID,
srm.SlsperID,
rd.SlsFreq,
StartDate = slr.StartDate,
EndDate = slr.EndDate

In [285]:
T_ExtRoute.shape
# df_filter(T_ExtRoute, SalesRouteID='TDN1_1', DayofWeek='Sun')
# df_filter(T_ExtRoute, SalesRouteID='TDN1_1').to_clipboard()
# vc(T_ExtRoute, 'SalesRouteID').to_clipboard()
# T_ExtRoute.shape
# T_ExtRoute.shape
# df_SRM.shape
# vc(df_T_RouteDet,'DayofWeek')
# vc(df_SRM, 'FromDate')

(6341, 6)

In [286]:
RD.dtypes

CustID                  object
SlsFreq                 object
DayofWeek               object
SalesRouteID            object
VisitDate       datetime64[ns]
dtype: object

In [287]:
# RD.VisitDate = convert_to_datetime(RD.VisitDate)

In [288]:
TResult_1 = RD.merge(SRM.add_prefix('SRM_'), how='inner', left_on=['SalesRouteID'], right_on='SRM_SalesRouteID')
TResult_1=TResult_1[\
    (TResult_1['DayofWeek'] !='Sun') & \
    (TResult_1['VisitDate'] >= TResult_1['SRM_FromDate']) & \
    (TResult_1['VisitDate'] <= TResult_1['SRM_ToDate'])]
lo(6)
TResult_1=TResult_1[['SRM_BranchID', 'SRM_SlsperID', 'SalesRouteID', 'SlsFreq', 'CustID', 'VisitDate']]
TResult_1['InRoute']=1
TResult_1['ExtRoute']=0
TResult_1['Visited']=0
TResult_1['OrderFromPDA']=0
TResult_1['OrdAmtFromPDA']=0
TResult_1['OrderFromOther']=0
TResult_1['OrdAmtFromOther']=0

In [289]:
TResult_1.columns = ['BranchID', 'SlsperID', 'SalesRouteID', 'SlsFreq', 'CustID',
       'VisitDate', 'InRoute', 'ExtRoute', 'Visited', 'OrderFromPDA',
       'OrdAmtFromPDA', 'OrderFromOther', 'OrdAmtFromOther']

In [290]:
TResult_1.columns

Index(['BranchID', 'SlsperID', 'SalesRouteID', 'SlsFreq', 'CustID',
       'VisitDate', 'InRoute', 'ExtRoute', 'Visited', 'OrderFromPDA',
       'OrdAmtFromPDA', 'OrderFromOther', 'OrdAmtFromOther'],
      dtype='object')

In [291]:
df_filter(TResult_1, BranchID='MR0016', SlsperID='MR0678')

,BranchID,SlsperID,SalesRouteID,SlsFreq,CustID,VisitDate,InRoute,ExtRoute,Visited,OrderFromPDA,OrdAmtFromPDA,OrderFromOther,OrdAmtFromOther
137764,MR0016,MR0678,TMK2_9,F2,000476,2021-10-11,1,0,0,0,0,0,0
137765,MR0016,MR0678,TMK2_9,A,000831,2021-10-01,1,0,0,0,0,0,0
137766,MR0016,MR0678,TMK2_9,A,000831,2021-10-02,1,0,0,0,0,0,0
137767,MR0016,MR0678,TMK2_9,A,000831,2021-10-04,1,0,0,0,0,0,0
137768,MR0016,MR0678,TMK2_9,A,000831,2021-10-05,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
140500,MR0016,MR0678,TMK2_9,A,P4406-0083,2021-10-19,1,0,0,0,0,0,0
140501,MR0016,MR0678,TMK2_9,A,P4406-0083,2021-10-20,1,0,0,0,0,0,0
140502,MR0016,MR0678,TMK2_9,A,P4406-0083,2021-10-21,1,0,0,0,0,0,0
140503,MR0016,MR0678,TMK2_9,A,P4406-0083,2021-10-22,1,0,0,0,0,0,0


In [292]:
TResult_1.shape
# stl = f"SELECT BranchID, SlsperID, CustID, UpdateTime, NumberCICO FROM dbo.AR_SalespersonLocationTrace WHERE CAST(UpdateTime AS DATE) BETWEEN {param_1} AND {param_2}"
# get_ms_csv(STL, 'STL.csv')
# STL = pd.read_csv('STL.csv')
# STL['UpdateTime']=pd.to_datetime(df_STL['UpdateTime'].str[:10], dayfirst=True)
# df_STL.shape


(244123, 13)

In [293]:
# Faster performance 2.4s
df1 = STL.merge(WCC.add_prefix('WCC_'), how='left', left_on=['BranchID', 'NumberCICO'], right_on=['WCC_BranchID','WCC_NumberCICO'])
df2 = df1.merge(SRM.add_prefix('SRM_'), how='inner', left_on=['BranchID','SlsperID'], right_on=['SRM_BranchID','SRM_SlsperID'])
df3=df2[\
    (df2['UpdateTime'] >= df2['SRM_FromDate']) & \
    (df2['UpdateTime'] <= df2['SRM_ToDate'])].copy()

_RD=RD[RD['DayofWeek']=='Sun'].copy()
# _RD['VisitDate']=convert_to_datetime(_RD['VisitDate'])
_RD['SalesRouteID'] = _RD['SalesRouteID'].str.upper()
# df_filter(_RD, SalesRouteID='THCM_12', CustID='M032201')
df3['SRM_SalesRouteID'] = df3['SRM_SalesRouteID'].str.upper()
df4 = df3.merge(_RD.add_prefix('RD_'), how='left', left_on=['CustID', 'SRM_SalesRouteID'], right_on=['RD_CustID','RD_SalesRouteID'])
print(df4.shape)
# df4.columns
ctr1=df4.RD_VisitDate<df4.SRM_FromDate
ctr2=df4.RD_VisitDate>df4.SRM_ToDate
series = df4['RD_SalesRouteID']
list_check = df3.columns.to_list()
df5 = date_between_handler(df4,ctr1,ctr2,series,list_check)
# print(df5.shape)
TResult_2=df_na(df5, 'WCC_NumberCICO')
# TResult_2.shape
TResult_2=TResult_2[['BranchID', 'SlsperID', 'RD_SalesRouteID', 'RD_SlsFreq', 'CustID', 'UpdateTime']]
# TResult_2.columns
TResult_2['InRoute']=0
TResult_2['ExtRoute']=np.where(TResult_2.RD_SalesRouteID.notna(), 1,0)
TResult_2['Visited']=1
TResult_2['OrderFromPDA']=0
TResult_2['OrdAmtFromPDA']=0
TResult_2['OrderFromOther']=0
TResult_2['OrdAmtFromOther']=0
TResult_2 = TResult_2.drop_duplicates()
TResult_2.columns=TResult_1.columns

(222146, 17)
row errors = 0
row errors keep = 0


In [294]:
TResult_2.shape
# list = PSO.columns.tolist()
# PSO.columns

(14260, 13)

In [295]:
ET = T_ExtRoute
# df1 = PSO.merge(ET.add_prefix('ET_'), how='left', left_on=['BranchID', 'CustID', 'SlsPerID'], right_on=['ET_BranchID', 'ET_CustID', 'ET_SlsperID'])
df1 = PSO.merge(SLR.add_prefix('SLR_'), how='inner', left_on=['CustID'], right_on=['SLR_CustID'])
ctr1=df1.Crtd_DateTime >= df1['SLR_StartDate']
ctr2=df1.Crtd_DateTime <= df1['SLR_EndDate']
df2=df1[ctr1 & ctr2].copy()

df3 = df2.merge(ET.add_prefix('ET_'), how='left', left_on=['BranchID', 'CustID', 'SlsPerID'], right_on=['ET_BranchID', 'ET_CustID', 'ET_SlsperID'])
ctr1 = df3['Crtd_DateTime'] < df3.ET_StartDate
ctr2 = df3['Crtd_DateTime'] > df3.ET_EndDate
series = df3['ET_BranchID']
list_check = df2.columns.tolist()
df3 = date_between_handler(df3, ctr1, ctr2, series, list_check)

# df2 = df1.merge(SLR.add_prefix('SLR_'), how='inner', left_on=['CustID'], right_on=['SLR_CustID'])


row errors = 117
row errors keep = 108


In [296]:
df3.columns

Index(['BranchID', 'SlsPerID', 'SalesRouteID', 'CustID', 'Crtd_DateTime',
       'InsertFrom', 'LineAmt', 'SLR_CustID', 'SLR_SalesRouteID',
       'SLR_StartDate', 'SLR_EndDate', 'SLR_SlsFreq', 'ET_BranchID',
       'ET_CustID', 'ET_SlsperID', 'ET_SlsFreq', 'ET_StartDate', 'ET_EndDate'],
      dtype='object')

In [297]:
ET = T_ExtRoute
# df1 = PSO.merge(ET.add_prefix('ET_'), how='left', left_on=['BranchID', 'CustID', 'SlsPerID'], right_on=['ET_BranchID', 'ET_CustID', 'ET_SlsperID'])
df1 = PSO.merge(SLR.add_prefix('SLR_'), how='inner', left_on=['CustID'], right_on=['SLR_CustID'])
ctr1=df1.Crtd_DateTime >= df1['SLR_StartDate']
ctr2=df1.Crtd_DateTime <= df1['SLR_EndDate']
df2=df1[ctr1 & ctr2].copy()

df3 = df2.merge(ET.add_prefix('ET_'), how='left', left_on=['BranchID', 'CustID', 'SlsPerID'], right_on=['ET_BranchID', 'ET_CustID', 'ET_SlsperID'])
ctr1 = df3['Crtd_DateTime'] < df3.ET_StartDate
ctr2 = df3['Crtd_DateTime'] > df3.ET_EndDate
series = df3['ET_BranchID']
list_check = df2.columns.tolist()
df3 = date_between_handler(df3, ctr1, ctr2, series, list_check)
# df3.shape
# df3.columns
df3['InRoute']=0
df3['ExtRoute']=np.where(df3.ET_BranchID.notna(), 1,0)
df3['Visited']=0
df3['OrderFromPDA']=np.where(df3['InsertFrom']=='S', 1, 0)
df3['OrdAmtFromPDA']=np.where(df3['InsertFrom']=='S', df3.LineAmt, 0)
df3['OrderFromOther']=np.where(df3['InsertFrom']!='S', 1, 0)
df3['OrdAmtFromOther']=np.where(df3['InsertFrom']!='S', df3.LineAmt, 0)
# lo(9)
grouplist = ['BranchID', 'SlsPerID', 'SalesRouteID', 'SLR_SlsFreq', 'CustID', 'Crtd_DateTime', 'ExtRoute', 'OrderFromPDA', 'OrderFromOther', 'InRoute', 'Visited']
agg_dict = {'OrdAmtFromPDA':np.sum, 'OrdAmtFromOther':np.sum}
TResult_3 = pivot(df3, grouplist, agg_dict)
TResult_3 = TResult_3[['BranchID', 'SlsPerID', 'SalesRouteID', 'SLR_SlsFreq', 'CustID', 'Crtd_DateTime', 'InRoute', 'ExtRoute', 'Visited', 'OrderFromPDA', 'OrdAmtFromPDA', 'OrderFromOther', 'OrdAmtFromOther']]
TResult_3.columns = TResult_1.columns
# TResult_1.columns
# TResult_3.columns

row errors = 117
row errors keep = 108


In [298]:
TResult_3.shape

(15365, 13)

In [299]:
TResult_1.dtypes

BranchID                   object
SlsperID                   object
SalesRouteID               object
SlsFreq                    object
CustID                     object
VisitDate          datetime64[ns]
InRoute                     int64
ExtRoute                    int64
Visited                     int64
OrderFromPDA                int64
OrdAmtFromPDA               int64
OrderFromOther              int64
OrdAmtFromOther             int64
dtype: object

In [300]:
TResult_2.dtypes

BranchID                   object
SlsperID                   object
SalesRouteID               object
SlsFreq                    object
CustID                     object
VisitDate          datetime64[ns]
InRoute                     int64
ExtRoute                    int32
Visited                     int64
OrderFromPDA                int64
OrdAmtFromPDA               int64
OrderFromOther              int64
OrdAmtFromOther             int64
dtype: object

In [301]:
TResult_3.dtypes

BranchID                   object
SlsperID                   object
SalesRouteID               object
SlsFreq                    object
CustID                     object
VisitDate          datetime64[ns]
InRoute                     int64
ExtRoute                    int64
Visited                     int64
OrderFromPDA                int64
OrdAmtFromPDA             float64
OrderFromOther              int64
OrdAmtFromOther           float64
dtype: object

In [302]:
TResult_4 = union_all([TResult_1, TResult_2])
TResult_4 = union_all([TResult_4, TResult_3])

In [303]:
TResult_4.dtypes

BranchID                   object
SlsperID                   object
SalesRouteID               object
SlsFreq                    object
CustID                     object
VisitDate          datetime64[ns]
InRoute                     int64
ExtRoute                    int64
Visited                     int64
OrderFromPDA                int64
OrdAmtFromPDA             float64
OrderFromOther              int64
OrdAmtFromOther           float64
dtype: object

In [304]:
df_filter(TResult_4, BranchID='MR0001', SlsperID='MR0499', SalesRouteID='THCM_12', CustID='M0101108', SlsFreq='A').to_clipboard()

In [305]:
grouplist = ['BranchID', 'SlsperID', 'SalesRouteID', 'SlsFreq', 'CustID', 'VisitDate']
agg_dict = {'InRoute':np.max, 'ExtRoute':np.max, 'Visited':np.max, 'OrderFromPDA':np.max, 'OrdAmtFromPDA':np.max, 'OrderFromOther':np.max, 'OrdAmtFromOther':np.max}
TResult = pivot(TResult_4, grouplist, agg_dict)

In [306]:
# , 'ExtRoute', 'OrderFromPDA', 'OrderFromOther', 'InRoute', 'Visited'
# TResult = 
TResult.shape

(270381, 13)

In [307]:
# df_filter(TResult, BranchID='MR0001', SlsperID='MR0499', SalesRouteID='THCM_12', CustID='M0101108', SlsFreq='A').to_clipboard()
# TResult.to_csv('TResult.csv')
# 277995-277484

In [308]:
TS.columns

Index(['BranchID', 'SlsperID', 'SupID', 'ASM', 'RSMID'], dtype='object')

In [309]:
TResult.shape

(270381, 13)

In [310]:
U.columns

Index(['UserName', 'FirstName', 'Position'], dtype='object')

In [311]:
df1 = TResult.merge(U.add_prefix('U_'), how='inner', left_on=['SlsperID'], right_on=['U_UserName'])

In [312]:
# df1.shape
# df1 = TResult.merge(TS, how='left', on=['BranchID' , 'SlsperID'], )
# U.columns

In [313]:
df2 = TResult.merge(TS, how='left', on=['BranchID' , 'SlsperID'])

In [314]:
df2.columns

Index(['BranchID', 'SlsperID', 'SalesRouteID', 'SlsFreq', 'CustID',
       'VisitDate', 'InRoute', 'ExtRoute', 'Visited', 'OrderFromPDA',
       'OrdAmtFromPDA', 'OrderFromOther', 'OrdAmtFromOther', 'SupID', 'ASM',
       'RSMID'],
      dtype='object')

In [315]:
df3 = df2.merge(U.add_prefix('sup_'), how='left', left_on='SupID', right_on='sup_UserName')

In [316]:
# df2.columns

In [317]:
df4 = df3.merge(U.add_prefix('asm_'), how='left', left_on='ASM', right_on='asm_UserName')

In [318]:
df5 = df4.merge(U.add_prefix('rsm_'), how='left', left_on='RSMID', right_on='rsm_UserName')

In [319]:
df6 = df5.merge(COM.add_prefix('com_'), how='inner', left_on='BranchID', right_on='com_CpnyID')

In [320]:
CU.columns

Index(['CustId', 'CustName', 'Channel', 'ShopType', 'HCOID', 'HCOTypeID',
       'ClassId', 'TerritoryDescr', 'StateDescr', 'DistrictDescr',
       'WardDescr'],
      dtype='object')

In [321]:
df7 = df6.merge(CU.add_prefix('cu_'), how='inner', left_on='CustID', right_on='cu_CustId')

In [323]:
df7

Index(['BranchID', 'SlsperID', 'SalesRouteID', 'SlsFreq', 'CustID',
       'VisitDate', 'InRoute', 'ExtRoute', 'Visited', 'OrderFromPDA',
       'OrdAmtFromPDA', 'OrderFromOther', 'OrdAmtFromOther', 'SupID', 'ASM',
       'RSMID', 'sup_UserName', 'sup_FirstName', 'sup_Position',
       'asm_UserName', 'asm_FirstName', 'asm_Position', 'rsm_UserName',
       'rsm_FirstName', 'rsm_Position', 'com_CpnyID', 'com_CpnyName',
       'cu_CustId', 'cu_CustName', 'cu_Channel', 'cu_ShopType', 'cu_HCOID',
       'cu_HCOTypeID', 'cu_ClassId', 'cu_TerritoryDescr', 'cu_StateDescr',
       'cu_DistrictDescr', 'cu_WardDescr'],
      dtype='object')